In [1]:
import os
import timeit

import cv2
from tensorflow.keras.backend import clear_session
import numpy as np
from PIL import Image
from scipy.stats import entropy, norm

import image_process as ip
import tools as tl
import tensorflow as tf
from tensorflow.keras.applications import Xception
import gc
import models
from tensorflow.keras.optimizers import Adam
from keras_bcnn.models import BayesianUNet2D
from matplotlib import pylab as plt
from scipy.stats import entropy
from tensorflow.keras.utils import plot_model
from keras_bcnn.models import MCSampler
import scipy.misc

Using TensorFlow backend.


In [2]:
data='melanoma'
mode='test'
data_num=1
method='unet_patch'
patch_size=160
test_slide_step=10
ite=10
n_class=3
reflect=False

In [3]:
def get_ytrue(mask_array, data):
        if data == "ips":
            height, width, _ = mask_array.shape
            good_label = ((mask_array[:,:,0] == 255)&
                          (mask_array[:,:,1] == 0)&
                          (mask_array[:,:,2] == 0)
                         ) * np.ones((height, width)) * 1
            bad_label = ((mask_array[:,:,0] == 0)&
                         (mask_array[:,:,1] == 255)&
                         (mask_array[:,:,2] == 0)
                        ) * np.ones((height, width)) * 2
            bgd_label = ((mask_array[:,:,0] == 0)&
                         (mask_array[:,:,1] == 0)&
                         (mask_array[:,:,2] == 255)
                        ) * np.ones((height, width)) * 3
            y_true = good_label + bad_label + bgd_label
            return y_true
        else:
            height, width = mask_array.shape
            y_true = (mask_array == 255) * np.ones((height, width))
            return y_true

In [4]:
def pre_un_load(path):
    """
    data: str, "ips" or "melanoma"
    mode: str, "train" or "test"
    d_num: int, 1, ..., 5
    """
    img_path=[]
    if data in ['ips', 'melanoma']:
        for x in os.listdir(path):
            img_path.append(path+x)
        return img_path

def pre_cu_load(path):
    """
    data: str, "ips" or "melanoma"
    mode: str, "train" or "test"
    d_num: int, 1, ..., 5
    """
    img_path=[]
    if data in ['ips', 'melanoma']:
        for x in os.listdir(path):
            img_path.append(path+x)
        return img_path

In [5]:
#ipsのuncertainty evaluation
a_tpr = []
a_npv = []
a_acc = []
model_name="unet"
mode="test"
for data_num in [1]:
    img_path, mask_path = tl.data_path_load(data,'test',data_num)
    path=("./%s/%s/%s/size_%d/dataset_%d/%s_un/correctness/" %(data, method,model_name,patch_size,data_num,mode))
    path2=("./%s/%s/%s/size_%d/dataset_%d/%s_un/uncertainty_T_0.5/" %(data, method,model_name,patch_size,data_num,mode))
    #path=("./%s/%s/%s/size_%d/dataset_%d/stage2/%s_un/correctness/" %(data, method,model_name,patch_size,data_num,mode))
    #path2=("./%s/%s/%s/size_%d/dataset_%d/stage2/%s_un/uncertainty_T_0.5/" %(data, method,model_name,patch_size,data_num,mode))
    #path=("./%s/%s/%s/size_%d/dataset_%d/stage2/stage3/stage4/%s_un/correctness/" %(data, method,model_name,patch_size,data_num,mode))
    #path2=("./%s/%s/%s/size_%d/dataset_%d/stage2/stage3/stage4/%s_un/uncertainty_T_0.5/" %(data, method,model_name,patch_size,data_num,mode))
    un_path = pre_un_load(path2)
    cu_path = pre_cu_load(path)
    mask_path.sort()
    un_path.sort()
    cu_path.sort()
    save_path = tl.get_save_path(data, method, 'unet',patch_size, data_num)
    save_path = save_path#+"stage2/"#stage3/stage4/"###############
    with open(save_path + "image_evaluate2.txt", mode = "a") as file:
        title = ["<<", method, ">>"]
        title = " ".join(title)
        file.write(title + "\n")
        file.write("image_name  NPV TPR acc. \n")
    all_acc=0
    all_npv=0
    all_tpr=0
    for m_path,u_path,c_path in zip(mask_path,un_path,cu_path):
        name = os.path.basename(m_path)[:-4]
        m_mask = cv2.imread(m_path)
        m_mask = cv2.cvtColor(m_mask, cv2.COLOR_BGR2RGB)
        height, width = m_mask.shape[:2]
        imim = m_mask[:,:,0]+m_mask[:,:,1]+m_mask[:,:,2]
        m_mask[:,:,0]=(imim==255)*m_mask[:,:,0]
        m_mask[:,:,1]=(imim==255)*m_mask[:,:,1]
        m_mask[:,:,2]=(imim==255)*m_mask[:,:,2]

        mask = get_ytrue(m_mask, 'ips')
        oor = ~(mask == 0) *1

        u_mask = cv2.imread(u_path)
        u=u_mask[:,:,0]*oor

        c_mask = cv2.imread(c_path)
        #print(c_path)
        c=c_mask[:,:,0]*oor

        TP=np.zeros((1200,1600))
        Fp=np.zeros((1200,1600))
        TN=np.zeros((1200,1600))
        FN=np.zeros((1200,1600))
        TP=np.where((u == 255) & (c == 255) ,1, 0)
        FP=np.where((u == 0) & (c == 255) ,1, 0)
        TN=np.where((u == 0) & (c == 0) ,1, 0)
        FN=np.where((u == 255) & (c == 0) ,1, 0)
        a=(np.count_nonzero(TP == 1)+np.count_nonzero(TN == 1))/(np.count_nonzero(TN == 1)+np.count_nonzero(FN == 1)+np.count_nonzero(TP == 1)+np.count_nonzero(FP == 1))
        n=np.count_nonzero(TN == 1)/(np.count_nonzero(TN == 1)+np.count_nonzero(FN == 1))
        t=np.count_nonzero(TP == 1)/(np.count_nonzero(TP == 1)+np.count_nonzero(FN == 1))
        all_acc+=a
        all_npv+=n
        all_tpr+=t

        with open(save_path + "image_evaluate2.txt", mode = "a") as file:
            results = name+" "+str(n)+" "+str(t)+" "+str(a)+"\n"
            file.write(results)
    all_acc=all_acc/len(mask_path)
    all_npv=all_npv/len(mask_path)
    all_tpr=all_tpr/len(mask_path)
    with open(save_path + "image_evaluate2.txt", mode = "a") as file:
            results = "all"+"      "+str(all_npv)+" "+str(all_tpr)+" "+str(all_acc)+"\n"
            file.write(results)
    with open(save_path + "image_evaluate2.txt") as f:
            s = f.readlines()
            all_result = s[-1]
            all_result =all_result.split()
            a_npv.append(float(all_result[1]))
            a_tpr.append(float(all_result[2]))
            a_acc.append(float(all_result[3]))
    #print(a_acc)
    tpr_a = np.average(a_tpr)
    npv_a = np.average(a_npv)
    acc_a = np.average(a_acc)
    tpr_s = np.std(a_tpr)
    npv_s = np.std(a_npv)
    acc_s = np.std(a_acc)
    with open("./%s/%s/%s/size_%d/uncertainty_result2.txt"%(data,
                                                       method,
                                                       model_name,
                                                       patch_size), "w") as file:
            file.write(str(tpr_a) + " ± " + str(tpr_s) + "\n")
            file.write(str(npv_a) + " ± " + str(npv_s) + "\n")
            file.write(str(acc_a) + " ± " + str(acc_s) + "\n")

ZeroDivisionError: division by zero

In [6]:
#melanomaのuncertainty evaluation
a_tpr = []
a_npv = []
a_acc = []
model_name="unet"
mode="test"
for data_num in [1,2,3,4,5]:
    print(data_num)
    img_path, mask_path = tl.data_path_load(data,'test',data_num)
    #path=("./%s/%s/%s/size_%d/dataset_%d/%s_un/correctness/" %(data, method,model_name,patch_size,data_num,mode))
    #path2=("./%s/%s/%s/size_%d/dataset_%d/%s_un/uncertainty_T_0.5/" %(data, method,model_name,patch_size,data_num,mode))
    path=("./%s/%s/%s/size_%d/dataset_%d/stage2/%s_un/correctness/" %(data, method,model_name,patch_size,data_num,mode))
    path2=("./%s/%s/%s/size_%d/dataset_%d/stage2/%s_un/uncertainty_T_0.5/" %(data, method,model_name,patch_size,data_num,mode))
    un_path = pre_un_load(path2)
    cu_path = pre_cu_load(path)
    mask_path.sort()
    un_path.sort()
    cu_path.sort()
    save_path = tl.get_save_path(data, method, 'unet',patch_size, data_num)
    save_path = save_path+"stage2/"
    with open(save_path + "image_evaluate2.txt", mode = "a") as file:
        title = ["<<", method, ">>"]
        title = " ".join(title)
        file.write(title + "\n")
        file.write("image_name  NPV TPR acc. \n")
    all_acc=0
    all_npv=0
    all_tpr=0
    for m_path,u_path,c_path in zip(mask_path,un_path,cu_path):
        name = os.path.basename(m_path)[:-4]
        m_mask = cv2.imread(m_path)
        m_mask = cv2.cvtColor(m_mask, cv2.COLOR_BGR2RGB)
        height, width = m_mask.shape[:2]

        u = cv2.imread(u_path)

        c = cv2.imread(c_path)
        #print(u_path)

        TP=np.zeros((height, width))
        Fp=np.zeros((height, width))
        TN=np.zeros((height, width))
        FN=np.zeros((height, width))
        TP=np.where((u == 255) & (c == 255) ,1, 0)
        FP=np.where((u == 0) & (c == 255) ,1, 0)
        TN=np.where((u == 0) & (c == 0) ,1, 0)
        FN=np.where((u == 255) & (c == 0) ,1, 0)
        a=(np.count_nonzero(TP == 1)+np.count_nonzero(TN == 1))/(np.count_nonzero(TN == 1)+np.count_nonzero(FN == 1)+np.count_nonzero(TP == 1)+np.count_nonzero(FP == 1))
        n=np.count_nonzero(TN == 1)/(np.count_nonzero(TN == 1)+np.count_nonzero(FN == 1))
        t=np.count_nonzero(TP == 1)/(np.count_nonzero(TP == 1)+np.count_nonzero(FN == 1))
        all_acc+=a
        all_npv+=n
        all_tpr+=t

        with open(save_path + "image_evaluate2.txt", mode = "a") as file:
            results = name+" "+str(n)+" "+str(t)+" "+str(a)+"\n"
            file.write(results)
    all_acc=all_acc/len(mask_path)
    all_npv=all_npv/len(mask_path)
    all_tpr=all_tpr/len(mask_path)
    with open(save_path + "image_evaluate2.txt", mode = "a") as file:
            results = "all"+"      "+str(all_npv)+" "+str(all_tpr)+" "+str(all_acc)+"\n"
            file.write(results)
    
    with open(save_path + "image_evaluate2.txt") as f:
            s = f.readlines()
            all_result = s[-1]
            all_result =all_result.split()
            a_npv.append(float(all_result[1]))
            a_tpr.append(float(all_result[2]))
            a_acc.append(float(all_result[3]))
    #print(a_acc)
    tpr_a = np.average(a_tpr)
    npv_a = np.average(a_npv)
    acc_a = np.average(a_acc)
    tpr_s = np.std(a_tpr)
    npv_s = np.std(a_npv)
    acc_s = np.std(a_acc)
    with open("./%s/%s/%s/size_%d/uncertainty_result_dice.txt"%(data,
                                                       method,
                                                       model_name,
                                                       patch_size), "w") as file:
            file.write(str(tpr_a) + " ± " + str(tpr_s) + "\n")
            file.write(str(npv_a) + " ± " + str(npv_s) + "\n")
            file.write(str(acc_a) + " ± " + str(acc_s) + "\n")

1
2
3
4
5
